# Modules and Grade_CMC

In [1]:
import numpy as np
import pandas as pd
import pymongo
import datetime
import cuemath_db as cuedb
import statistics
# Grade_module = pd.read_csv(r"C:\Users\yeshwanth\Desktop\Modules and MA_CMC - Modules.csv")

In [2]:
def get_foundation_module_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'code', 'is_assessment_live']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_module;"
    cur.execute(sql_query)
    df_module_id_code = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_module_id_code)

def get_foundation_student_attempt_data(module_list):
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'type', 'status', 'responses', 'score_details', 'module_id', 'student_id', 'objective_responses']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_studentassessment where module_id=ANY(%s);"
    cur.execute(sql_query, [module_list,])
    df_assessment_attempt = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_assessment_attempt)

def get_foundation_student_ma_data(segment_id):
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'type', 'assessment_response', 'student_grade', 'skill_segment_id', 'student_id']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_studentskillsegmentattempt where skill_segment_id=(%s);"
    cur.execute(sql_query, [segment_id, ])
    df_ma_attempt = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_ma_attempt)

def get_foundation_skillsegment_data(segment_list):
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    skillseg_cols = ['id', 'name', 'segment_code']
    skill_cols = ['id', 'name', 'skill_code']
    sql_query = "SELECT "+", ".join(["curriculum_skillsegment." + s for s in skillseg_cols])+ ", "+ " ,".join(["curriculum_skill." + s for s in skill_cols]) + " FROM curriculum_skillsegment inner join curriculum_skill on curriculum_skillsegment.skill_id = curriculum_skill.id where curriculum_skillsegment.id=ANY(%s);"    
    cur.execute(sql_query, [segment_list,])
    df_module_id_code = pd.DataFrame(cur.fetchall(), columns=['segment.' + s for s in skillseg_cols] + \
                                                             ['skill.' + s for s in skill_cols])
    cur.close()
    conn.close()
    return(df_module_id_code)

def get_question_ids_from_code():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    sql_query = "select id, code from content_templatequestion;"
    cur.execute(sql_query)
    df = pd.DataFrame(cur.fetchall(), columns=['id', 'code'])
    cur.close()
    conn.close()
    return(df)


In [3]:
import json
def fetch(row, data):
#     print("DEBUG", data)
    a = row["assessment_response"]
    c=json.loads(a)
    #     print(c)
    if(c.get('response', None) is not None):
        b = c['response']
    elif(c.get('responses', None) is not None):
        b = c['responses']
    else:
        print("KEYERROR", c)
    module_id=row["skill_segment_id"]
    student_id=row["student_id"]
    if(data.get(module_id, None) is None):
        data.update({module_id:{}})
    for j in range(len(b)):
        if(b[j].get("code", None) is not None):
            q_code =b[j]["code"]
        elif(b[j].get("id", None) is not None):
            q_code =b[j]["id"]
        else:
            continue
        if(data[module_id].get(row['student_grade']) is not None):
            if(data[module_id][row['student_grade']].get(q_code, None) is not None):
                data[module_id][row['student_grade']][q_code].update({student_id:{"time":b[j]["time"],"result":b[j]["result"],"type":row["type"],"answer":b[j]["answer"]}})
            else:
                data[module_id][row['student_grade']].update({q_code:{student_id:{"time":b[j]["time"],"result":b[j]["result"],"type":row["type"],"answer":b[j]["answer"]}}})
        else:
            data[module_id][row['student_grade']] = {q_code:{student_id:{"time":b[j]["time"],"result":b[j]["result"],"type":row["type"],"answer":b[j]["answer"]}}}
    return data


In [8]:
ques_dict = {}
ques_set = set()
skill_segment_lst = pd.read_csv('./Foundation_Data/Skill Segments PTS.csv')
# print(list(skill_segment_lst['id']))
df_unique_skillsegments = get_foundation_skillsegment_data(list(skill_segment_lst['id']))
# print(df_unique_skillsegments)
for i in range(df_unique_skillsegments.shape[0]):
    #for i in range(0,1):
    print("At Skill Num: ", i)
    df_skill_attempt = get_foundation_student_ma_data(df_unique_skillsegments.iloc[i]['segment.id'])
    print("DATA FETCHED")
    for j in range(df_skill_attempt.shape[0]):
        ques_dict = fetch(df_skill_attempt.iloc[j], ques_dict)
        # ques_dict.update(attempt_row)
df_template_question_id = get_question_ids_from_code()
print("DONE", len(ques_dict))


At Skill Num:  0
DATA FETCHED
At Skill Num:  1
DATA FETCHED
At Skill Num:  2
DATA FETCHED
At Skill Num:  3
DATA FETCHED
At Skill Num:  4
DATA FETCHED
At Skill Num:  5
DATA FETCHED
At Skill Num:  6
DATA FETCHED
At Skill Num:  7
DATA FETCHED
At Skill Num:  8
DATA FETCHED
At Skill Num:  9
DATA FETCHED
At Skill Num:  10
DATA FETCHED
At Skill Num:  11
DATA FETCHED
At Skill Num:  12
DATA FETCHED
At Skill Num:  13
DATA FETCHED
At Skill Num:  14
DATA FETCHED
At Skill Num:  15
DATA FETCHED
At Skill Num:  16
DATA FETCHED
At Skill Num:  17
DATA FETCHED
At Skill Num:  18
DATA FETCHED
At Skill Num:  19
DATA FETCHED
At Skill Num:  20
DATA FETCHED
At Skill Num:  21
DATA FETCHED
At Skill Num:  22
DATA FETCHED
At Skill Num:  23
DATA FETCHED
At Skill Num:  24
DATA FETCHED
At Skill Num:  25
DATA FETCHED
At Skill Num:  26
DATA FETCHED
At Skill Num:  27
DATA FETCHED
At Skill Num:  28
DATA FETCHED
At Skill Num:  29
DATA FETCHED
At Skill Num:  30
DATA FETCHED
At Skill Num:  31
DATA FETCHED
At Skill Num:  32


In [ ]:
df_template_question_id = get_question_ids_from_code()


In [11]:
fl_nm = './Foundation_data/out_skill_segments_pts.csv'
with open(fl_nm, "w") as fp:
    fp.write('skill_code, skill_name, skill_segment_name, skill_segment_code, Student_grade, Question Code, Question_URL, Num Attempts, Num Correct, Num Incorrect, Accuracy, Time if Correct, Time if Incorrect \n')
for module_id, module_obj in ques_dict.items():
    for student_grade, inner_obj in module_obj.items():
        for q_code, ques_obj in inner_obj.items():
            skill_segment_row = df_unique_skillsegments[df_unique_skillsegments['segment.id']==module_id].iloc[0]
            skill_name = skill_segment_row['skill.name']
            skill_segment_name = skill_segment_row['segment.name']
            skill_segment_code = skill_segment_row['segment.segment_code']
            skill_code = skill_segment_row['skill.skill_code']
            template_q_row = df_template_question_id[df_template_question_id['code'] == q_code]
            if(template_q_row.shape[0] != 0):
                question_url = "https://crm.cuemath.com/admin/content/templatequestion/" + template_q_row.iloc[0]['id']
            else:
                question_url = "https://crm.cuemath.com/admin/content/question/?q=" + q_code
            aggregate_question_data(skill_code, skill_name, skill_segment_name, skill_segment_code, question_url, student_grade, q_code, ques_obj)
     

MA.D1.C1.K48.S1 MA.C1.K48 I Compare_numbers_41 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_41 2182 1666 516 0.763519706691109 0.01041042136854742 0.007876606589147284 178 1666 170 168 0.009811224106324479
MA.D1.C1.K48.S1 MA.C1.K48 I Compare_numbers_57 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_57 2153 1715 438 0.7965629354389224 0.011985574344023335 0.008265787671232887 101 1715 197 140 0.01122883186251742
MA.D1.C1.K48.S1 MA.C1.K48 I Compare_numbers_28 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_28 2225 1777 448 0.7986516853932585 0.010528572312886883 0.0075384754464285775 162 170 116 1777 0.0099265213483146
MA.D1.C1.K48.S1 MA.C1.K48 I Compare_numbers_6 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_6 2207 1821 386 0.8251019483461712 0.009548290499725415 0.007291759067357513 134 1821 101 151 0.009153627548708647
MA.D1.C1.K48.S1 MA.C1.K48 I Compare_numbers_32 https://crm.cuemath.com/admin/content/quest

MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_57 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_57 1299 922 377 0.7097767513471901 0.014010348156182222 0.008632302387267896 92 922 143 142 0.012449514241724418
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_30 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_30 1268 884 384 0.6971608832807571 0.014327435520362003 0.008500692708333339 133 884 121 130 0.012562869873817046
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_18 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_18 1258 809 449 0.6430842607313195 0.013487284301606929 0.009490763919821824 102 809 137 210 0.01206086327503973
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_26 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_26 1249 858 391 0.6869495596477182 0.014914496503496518 0.009626572890025569 86 858 108 197 0.0132591096877502
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_9 https://crm.cuemath.com/admin/content/que

MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_59 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_59 1307 896 411 0.685539403213466 0.014874879464285713 0.008694912408759121 90 179 142 896 0.012931523335883705
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_58 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_58 1273 942 331 0.7399842890809112 0.014694945859872589 0.008062827794561932 71 116 942 144 0.012970490966221522
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_24 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_24 1259 963 296 0.7648927720413026 0.013304063343717549 0.009060182432432428 96 85 115 963 0.012306296266878469
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_13 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_13 1259 931 328 0.7394757744241461 0.014019212674543494 0.007031701219512197 931 87 137 104 0.012198796664019053
MA.D1.C1.K48.S1 MA.C1.K48 UKG Compare_numbers_32 https://crm.cuemath.com/admin/content/que

MA.D1.C1.K48.S1 MA.C1.K48 II Compare_numbers_7 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_7 115 103 12 0.8956521739130435 0.00877402912621359 0.004252916666666666 3 2 103 7 0.008302260869565216
MA.D1.C1.K48.S1 MA.C1.K48 II Compare_numbers_11 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_11 114 102 12 0.8947368421052632 0.009574392156862746 0.00338725 3 5 102 4 0.008923114035087722
MA.D1.C1.K48.S1 MA.C1.K48 II Compare_numbers_42 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_42 110 97 13 0.8818181818181818 0.0093529587628866 0.003493923076923077 3 2 97 8 0.008660527272727274
MA.D1.C1.K48.S1 MA.C1.K48 II Compare_numbers_12 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_12 101 84 17 0.8316831683168316 0.010416107142857145 0.007149411764705882 7 5 5 84 0.009866267326732675
MA.D1.C1.K48.S1 MA.C1.K48 II Compare_numbers_34 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_34 115 101 14 0.878260869

MA.D1.C1.K48.S1 MA.C1.K48 III Compare_numbers_53 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_53 1 1 0 1.0 0.0069169999999999995 NA 0 1 0 0 0.0069169999999999995
MA.D1.C1.K48.S1 MA.C1.K48 III Compare_numbers_48 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_48 1 1 0 1.0 0.012728999999999999 NA 1 0 0 0 0.012728999999999999
MA.D1.C1.K48.S1 MA.C1.K48 III Compare_numbers_2 https://crm.cuemath.com/admin/content/question/?q=Compare_numbers_2 1 1 0 1.0 0.016454999999999997 NA 0 1 0 0 0.016454999999999997
MA.D0.C1.K1.S1 MA.C1.K1 UKG Dominos_DL0_RcBZt8v_38 https://crm.cuemath.com/admin/content/question/?q=Dominos_DL0_RcBZt8v_38 1372 1151 221 0.8389212827988338 0.01256290008688096 0.010950022624434385 54 119 1151 48 0.012303099854227395
MA.D0.C1.K1.S1 MA.C1.K1 UKG Dominos_DL0_RcBZt8v_36 https://crm.cuemath.com/admin/content/question/?q=Dominos_DL0_RcBZt8v_36 1417 1191 226 0.8405081157374735 0.01229203778337533 0.010326172566371679 1191 126 47 53 0.01197849

...

MA.D7.C1.K48.S1 MA.C1.K48 VI MA_CAL_MISSINGOPS_DL7_MULSUB_230 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_MULSUB_230 2 2 0 1.0 0.006364 NA 0 0 2 0 0.006364
MA.D7.C1.K48.S1 MA.C1.K48 VI MA_CAL_MISSINGOPS_DL7_ADDSUB_161 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_ADDSUB_161 3 1 2 0.3333333333333333 0.008137 0.0027095 1 0 0 2 0.0045186666666666665
MA.D7.C1.K48.S1 MA.C1.K48 VI MA_CAL_MISSINGOPS_DL7_MULADD_193 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_MULADD_193 2 2 0 1.0 0.004437999999999999 NA 0 2 0 0 0.004437999999999999
MA.D7.C1.K48.S1 MA.C1.K48 VI MA_CAL_MISSINGOPS_DL7_DIVADD_260 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_DIVADD_260 2 2 0 1.0 0.0028735 NA 0 0 0 2 0.0028735
MA.D7.C1.K48.S1 MA.C1.K48 VI MA_CAL_MISSINGOPS_DL7_ADDSUB_166 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_ADDSUB_166 1 1 0 1.0 0.0054930000000000005 NA 1 0 0 0 0.0054930000000

MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_ADDSUB_174 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_ADDSUB_174 1 0 1 0.0 NA 0.021977 0 0 0 1 0.021977
MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_MULADD_199 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_MULADD_199 1 1 0 1.0 0.011544 NA 0 1 0 0 0.011544
MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_ADDSUB_172 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_ADDSUB_172 1 0 1 0.0 NA 0.00325 0 0 0 1 0.00325
MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_MULADD_186 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_MULADD_186 1 1 0 1.0 0.012265 NA 0 1 0 0 0.012265
MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_MULADD_194 https://crm.cuemath.com/admin/content/question/?q=MA_CAL_MISSINGOPS_DL7_MULADD_194 1 1 0 1.0 0.004567 NA 0 1 0 0 0.004567
MA.D7.C1.K48.S1 MA.C1.K48 IV MA_CAL_MISSINGOPS_DL7_MULADD_196 https://crm.cuemath.

MA.D5.C1.K3.S3 MA.C1.K3 IV 2MUL33 https://crm.cuemath.com/admin/content/templatequestion/08eb2fe4-4e8a-11e6-a1a4-acbc3277df8f 227 192 35 0.8458149779735683 0.008264552083333335 0.008785828571428572 0 0 0 0 0.00834492511013216
MA.D5.C1.K3.S3 MA.C1.K3 IV 2MUL28 https://crm.cuemath.com/admin/content/templatequestion/08029fcc-4e8a-11e6-8d04-acbc3277df8f 231 206 25 0.8917748917748918 0.007369699029126218 0.00897176 0 0 0 0 0.007543082251082254
MA.D5.C1.K3.S3 MA.C1.K3 IV 2MUL22 https://crm.cuemath.com/admin/content/templatequestion/06e91f6c-4e8a-11e6-b061-acbc3277df8f 230 219 11 0.9521739130434783 0.004880520547945207 0.007053363636363635 0 0 0 0 0.004984439130434782
MA.D5.C1.K3.S3 MA.C1.K3 IV 2MUL26 https://crm.cuemath.com/admin/content/templatequestion/07a3ba34-4e8a-11e6-ad8a-acbc3277df8f 226 212 14 0.9380530973451328 0.004663150943396226 0.007224285714285713 0 0 0 0 0.004821805309734512
MA.D5.C1.K3.S3 MA.C1.K3 VII 2MUL28 https://crm.cuemath.com/admin/content/templatequestion/08029fcc-4e8a

MA.D5.C1.K3.S3 MA.C1.K3 III 2MUL28 https://crm.cuemath.com/admin/content/templatequestion/08029fcc-4e8a-11e6-8d04-acbc3277df8f 25 22 3 0.88 0.007756727272727271 0.003706666666666667 0 0 0 0 0.00727072
MA.D5.C1.K3.S3 MA.C1.K3 II 2MUL38 https://crm.cuemath.com/admin/content/templatequestion/09d2ee80-4e8a-11e6-82d0-acbc3277df8f 4 4 0 1.0 0.00891775 NA 0 0 0 0 0.00891775
MA.D5.C1.K3.S3 MA.C1.K3 II 2MUL31 https://crm.cuemath.com/admin/content/templatequestion/088ea1b6-4e8a-11e6-80cc-acbc3277df8f 4 4 0 1.0 0.0076245 NA 0 0 0 0 0.0076245
MA.D5.C1.K3.S3 MA.C1.K3 II 2MUL26 https://crm.cuemath.com/admin/content/templatequestion/07a3ba34-4e8a-11e6-ad8a-acbc3277df8f 4 4 0 1.0 0.00971875 NA 0 0 0 0 0.00971875
MA.D5.C1.K3.S3 MA.C1.K3 II 2MUL32 https://crm.cuemath.com/admin/content/templatequestion/08bcd0d4-4e8a-11e6-a1e6-acbc3277df8f 4 4 0 1.0 0.0054705 NA 0 0 0 0 0.0054705
MA.D5.C1.K3.S3 MA.C1.K3 II 2MUL33 https://crm.cuemath.com/admin/content/templatequestion/08eb2fe4-4e8a-11e6-a1a4-acbc3277df8f 4

MA.D7.C1.K3.S3 MA.C1.K3 IV 2MUL37 https://crm.cuemath.com/admin/content/templatequestion/09a4e06c-4e8a-11e6-821a-acbc3277df8f 2 2 0 1.0 0.004336 NA 0 0 0 0 0.004336


In [14]:
with open('./Foundation_data/skill_ques_data.json', 'w') as outfile:
    json.dump(ques_dict, outfile)

In [9]:

def aggregate_question_data(skill_code, skill_name, skill_segment_name, skill_segment_code, ques_url, student_grade, ques_code, ques_data):
    num_attempts = 0
    #num_cleared_sess_cleared = 0
    #num_cleared_sess_failed = 0
    total_time = 0
    num_correct = 0
    num_incorrect = 0 
    num_A = 0
    num_B = 0 
    num_C = 0
    num_D = 0
    total_time_if_correct = 0
    total_time_if_incorrect = 0
    for student_id, dat in ques_data.items():
        dat['time'] = dat['time']/1000.0
        num_attempts += 1
        total_time += dat['time']
        if(dat['result']):
            num_correct += 1
            total_time_if_correct += dat['time']
            #if(dat['status']=='CLRD'):
            #    num_cleared_sess_cleared += 1
            #else:
            #    num_cleared_sess_failed += 1
        else:
            num_incorrect += 1
            total_time_if_incorrect += dat['time']
        if(dat['answer']=="A"):
            num_A +=1
        elif(dat['answer']=="B"):
            num_B +=1
        elif(dat['answer']=="C"):
            num_C +=1
        elif(dat['answer']=="D"):
            num_D +=1
    if(num_correct!=0):
        avg_time_if_correct = total_time_if_correct/num_correct
    else:
        avg_time_if_correct = "NA"
    if(num_incorrect!=0):
        avg_time_if_incorrect = total_time_if_incorrect/num_incorrect
    else:
        avg_time_if_incorrect = "NA"
    avg_time = total_time/num_attempts
    accuracy = num_correct/num_attempts
    print(skill_segment_code, skill_code, student_grade, ques_code, ques_url, num_attempts, num_correct, num_incorrect, accuracy, avg_time_if_correct,avg_time_if_incorrect, num_A,num_B,num_C,num_D,avg_time )
    out_list = [skill_code, skill_name, skill_segment_name, skill_segment_code, student_grade, ques_code, ques_url, num_attempts, num_correct, num_incorrect, accuracy, avg_time_if_correct, avg_time_if_incorrect]
    out_final_list = [str(a) for a in out_list]
    with open(fl_nm, "a") as fp:
        fp.write(", ".join(out_final_list) + '\n')

In [84]:
aggregate_question_data